In [1]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model, load_model
import numpy as np
from keras.applications.vgg16 import preprocess_input, decode_predictions


In [2]:
img_width, img_height = 150, 150
train_data_dir = 'd:/BSU-Projects/6th-Term/Neural-Net/Lab_9/data/train'
validation_data_dir = 'd:/BSU-Projects/6th-Term/Neural-Net/Lab_9/data/validation'
nb_train_samples = 32
nb_validation_samples = 8
epochs = 100
batch_size = 16

In [3]:
base_model = applications.VGG16(weights='imagenet', include_top=False)

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# Выходной слой с двумя нейронами для классов "кот" и "собака"
predictions = Dense(2, activation='softmax')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
# "Замораживаем" сверточные уровни сети VGG16
# Обучаем только вновь добавленные слои
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Компилируем модель
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Создаем генератор данных для обучения
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode= 'categorical')

# Создаем генератор данных для валидации
validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode= 'categorical')

Found 8 images belonging to 2 classes.


Found 2 images belonging to 2 classes.


In [9]:
# Обучаем модель с помощью генератора
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = nb_validation_samples)

Epoch 1/100


c:\Users\tr07s\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/32 ━━━━━━━━━━━━━━━━━━━━ 1:27 3s/step - accuracy: 0.5000 - loss: 0.7004

c:\Users\tr07s\AppData\Local\Programs\Python\Python39\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.5000 - loss: 0.7004 - val_accuracy: 0.5000 - val_loss: 2.4712
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5000 - loss: 1.9224 - val_accuracy: 0.5000 - val_loss: 1.3057
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5000 - loss: 1.3337 - val_accuracy: 0.5000 - val_loss: 1.2683
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7500 - loss: 0.5537 - val_accuracy: 1.0000 - val_loss: 0.5183
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 1.0000 - loss: 0.2502 - val_accuracy: 0.5000 - val_loss: 0.7118
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - loss: 0.1970 - val_accuracy: 0.5000 - val_loss: 0.5345
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.1616 - val_accuracy: 0.5000 - val_loss: 0.6099
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.1410 - val_accuracy: 0.5000 - val_loss: 0

In [10]:
print("Сохраняем сеть")
model.save('D:/BSU-Projects/6th-Term/Neural-Net/Lab_9/models/vgg16.keras')
print("Сохранение сети завершено")

Сохраняем сеть
Сохранение сети завершено


In [11]:
model = load_model('D:/BSU-Projects/6th-Term/Neural-Net/Lab_9/models/vgg16.keras')

In [12]:
img = image.load_img('D:/BSU-Projects/6th-Term/Neural-Net/Lab_9/data/train/cats/cat.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)

In [ ]:
print('Результаты распознавания:', preds[0])

Результаты распознавания: [2.1622661e-06 9.9999785e-01]
